In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# Try to import Moirai
try:
    from uni2ts.model.moirai import MoiraiForecast
    from uni2ts.model.moirai.module import MoiraiModule
    MOIRAI_AVAILABLE = True
except ImportError:
    print("Warning: uni2ts not available. Moirai model cannot be used.")
    MOIRAI_AVAILABLE = False


In [ ]:
SEQ_LEN = 30          # lookback window length
BATCH_SIZE = 64
HIDDEN_DIM = 64
NUM_EPOCHS = 30
LR = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
df = pd.read_csv("data/SHEL_data.csv")


In [ ]:

feature_cols = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]
values = df[feature_cols].astype(np.float32).values
n_samples, n_features = values.shape


In [ ]:
split_idx = int(0.9 * n_samples)


In [ ]:
scaler = StandardScaler()
values_train = values[:split_idx]
scaler.fit(values_train)
values_scaled = scaler.transform(values).astype(np.float32)


In [ ]:
def make_sequences(values_scaled, seq_len, split_idx):
    X_train, y_train = [], []
    X_test, y_test = [], []

    # training sequences: target index t in [seq_len, split_idx-1]
    for t in range(seq_len, split_idx):
        X_train.append(values_scaled[t-seq_len:t])
        y_train.append(values_scaled[t])

    # test sequences: target index t in [split_idx, n_samples-1]
    n_total = values_scaled.shape[0]
    for t in range(split_idx, n_total):
        if t - seq_len < 0:
            continue
        X_test.append(values_scaled[t-seq_len:t])
        y_test.append(values_scaled[t])

    X_train = np.stack(X_train)  # [N_train, T, D]
    y_train = np.stack(y_train)  # [N_train, D]
    X_test = np.stack(X_test)    # [N_test, T, D]
    y_test = np.stack(y_test)    # [N_test, D]

    return X_train, y_train, X_test, y_test


In [ ]:

X_train, y_train, X_test, y_test = make_sequences(values_scaled, SEQ_LEN, split_idx)


In [ ]:
print("Train sequences:", X_train.shape, "Test sequences:", X_test.shape)


In [ ]:
def compute_metrics(y_true_scaled, y_pred_scaled, X_test_scaled_last):
    """
    y_true_scaled: [N, D]
    y_pred_scaled: [N, D]
    X_test_scaled_last: [N, D]  (last input of each window, scaled)
    """
    # Scaled RMSE
    mse_scaled = np.mean((y_pred_scaled - y_true_scaled) ** 2)
    rmse_scaled = np.sqrt(mse_scaled)

    # MAPE (on original scale)
    y_true = scaler.inverse_transform(y_true_scaled)
    y_pred = scaler.inverse_transform(y_pred_scaled)
    eps = 1e-6
    mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + eps))) * 100.0

    # Mean Directional Accuracy (direction of change from last input to target)
    # use scaled values (sign preserved)
    last = X_test_scaled_last
    actual_change = np.sign(y_true_scaled - last)
    pred_change = np.sign(y_pred_scaled - last)
    correct_dir = (actual_change == pred_change).astype(np.float32)
    mda = correct_dir.mean()

    # R^2 (on scaled)
    r2 = r2_score(
        y_true_scaled.reshape(-1),
        y_pred_scaled.reshape(-1)
    )

    # Forecast bias (on original)
    bias_per_feature = np.mean(y_pred - y_true, axis=0)  # [D]
    bias_overall = bias_per_feature.mean()
    # classify: over / under / none
    avg_abs_level = np.mean(np.abs(y_true))
    threshold = 0.01 * avg_abs_level  # 1% of average magnitude
    if abs(bias_overall) < threshold:
        bias_flag = "None / minimal"
    elif bias_overall > 0:
        bias_flag = "Over-forecast (too high)"
    else:
        bias_flag = "Under-forecast (too low)"

    metrics = {
        "Scaled_RMSE": float(rmse_scaled),
        "MAPE_percent": float(mape),
        "MDA": float(mda),
        "R2": float(r2),
        "Bias_overall": float(bias_overall),
        "Bias_flag": bias_flag,
        "Bias_per_feature": dict(zip(feature_cols, bias_per_feature))
    }
    return metrics


In [ ]:
row_for_sheet = {
    "Model": model_name,
    "Scaled RMSE": metrics["Scaled_RMSE"],
    "MAPE": metrics["MAPE_percent"],
    "Mean Directional Accuracy (MDA)": metrics["MDA"],
    "R^2": metrics["R2"],
    "Forecast Bias?": metrics["Bias_flag"],
}
print("\nRow for spreadsheet:", row_for_sheet)


**SALESFORCE MOIRAI MODEL**


In [ ]:
model_name = "Salesforce Morai Model"

if not MOIRAI_AVAILABLE:
    print("Moirai model not available. Please install uni2ts: pip install uni2ts")
    print("For now, using a placeholder that will show the expected format.")
    
    # Placeholder predictions (mean of last timestep)
    all_preds_scaled = []
    all_true_scaled = []
    all_last_inputs_scaled = []
    
    for i in range(len(X_test)):
        all_preds_scaled.append(X_test[i, -1, :])  # Use last timestep as prediction
        all_true_scaled.append(y_test[i])
        all_last_inputs_scaled.append(X_test[i, -1, :])
    
    y_pred_scaled = np.array(all_preds_scaled)
    y_true_scaled = np.array(all_true_scaled)
    X_last_scaled = np.array(all_last_inputs_scaled)
    
else:
    # Use Moirai model
    try:
        # Moirai works with univariate series, so we'll process each feature separately
        # and combine predictions
        all_preds_scaled = []
        all_true_scaled = []
        all_last_inputs_scaled = []
        
        # Initialize Moirai model (using small version for efficiency)
        moirai_model = MoiraiForecast(
            module=MoiraiModule.from_pretrained("Salesforce/moirai-1.0-R-small"),
            prediction_length=1,
            context_length=SEQ_LEN,
            target_dim=1,
            feat_dynamic_real_dim=0,
            past_feat_dynamic_real_dim=0,
        )
        
        predictor = moirai_model.create_predictor(batch_size=BATCH_SIZE)
        
        # Process each test sample
        for i in range(len(X_test)):
            # Moirai expects format: [T] for univariate
            # We'll average across features or use first feature as proxy
            # For multivariate, we process each feature separately
            preds_per_feature = []
            
            for feat_idx in range(n_features):
                # Extract univariate series for this feature
                univariate_series = X_test[i, :, feat_idx]  # [T]
                
                # Convert to Moirai format (list of dicts)
                from gluonts.dataset.common import ListDataset
                test_data = ListDataset(
                    [{"target": univariate_series, "start": pd.Timestamp("2020-01-01")}],
                    freq="D"
                )
                
                # Get prediction
                forecasts = list(predictor.predict(test_data))
                if len(forecasts) > 0:
                    pred_value = forecasts[0].mean[0]  # Get mean prediction
                else:
                    pred_value = univariate_series[-1]  # Fallback to last value
                
                preds_per_feature.append(pred_value)
            
            all_preds_scaled.append(np.array(preds_per_feature))
            all_true_scaled.append(y_test[i])
            all_last_inputs_scaled.append(X_test[i, -1, :])
        
        y_pred_scaled = np.array(all_preds_scaled)
        y_true_scaled = np.array(all_true_scaled)
        X_last_scaled = np.array(all_last_inputs_scaled)
        
    except Exception as e:
        print(f"Error using Moirai model: {e}")
        print("Falling back to simple baseline (last timestep)")
        
        # Fallback: use last timestep as prediction
        all_preds_scaled = []
        all_true_scaled = []
        all_last_inputs_scaled = []
        
        for i in range(len(X_test)):
            all_preds_scaled.append(X_test[i, -1, :])
            all_true_scaled.append(y_test[i])
            all_last_inputs_scaled.append(X_test[i, -1, :])
        
        y_pred_scaled = np.array(all_preds_scaled)
        y_true_scaled = np.array(all_true_scaled)
        X_last_scaled = np.array(all_last_inputs_scaled)


In [ ]:
metrics = compute_metrics(y_true_scaled, y_pred_scaled, X_last_scaled)
print("\n=== Metrics for", model_name, "===")
for k, v in metrics.items():
    if isinstance(v, dict):
        print(k, ":")
        for kk, vv in v.items():
            print(f"  {kk}: {vv:.4f}")
    else:
        print(k, ":", v)


In [ ]:
row_for_sheet = {
    "Model": model_name,
    "Scaled RMSE": metrics["Scaled_RMSE"],
    "MAPE": metrics["MAPE_percent"],
    "Mean Directional Accuracy (MDA)": metrics["MDA"],
    "R^2": metrics["R2"],
    "Forecast Bias?": metrics["Bias_flag"],
}
print("\nRow for spreadsheet:", row_for_sheet)


In [ ]:
y_true = scaler.inverse_transform(y_true_scaled)
y_pred = scaler.inverse_transform(y_pred_scaled)

n_test = y_true.shape[0]
t_axis = np.arange(n_test)

fig, axes = plt.subplots(n_features, 1,
                         figsize=(12, 3 * n_features),
                         sharex=True)

if n_features == 1:
    axes = [axes]

for d, col in enumerate(feature_cols):
    ax = axes[d]
    ax.plot(t_axis, y_true[:, d], label=f"True {col}")
    ax.plot(t_axis, y_pred[:, d], label=f"Pred {col}")
    ax.set_ylabel(col)
    ax.grid(True, alpha=0.3)
    if d == 0:
        ax.legend(loc="upper right")

axes[-1].set_xlabel("Test window index (time)")
plt.suptitle(f"{model_name}: True vs Pred on Holdout (last 10%)")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
